In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import pandas as pd
import pickle

In [2]:
file = "predict_results_1biLSTM"
df = pd.read_csv("predict_res/"+file+".csv")
df.head(5)

,tweet_text,true_label,health,vax,gardasil,hpvvaccin,cancer,vaccin,cervicalcanc,studi,learntherisk,hpv,vaccineswork
0,vaccine cuts teen girls ' hpv rates by two-thi...,health,0.997747,0.001123,6.564490e-07,2.698930e-07,3.316900e-07,0.001125,6.192670e-07,1.152250e-06,4.404900e-07,8.083010e-07,1.055320e-07
1,"rt : . vaxxed update ! gardasil tragedy , lead...",hpv,0.000021,0.000244,5.348350e-01,2.528250e-05,7.599710e-06,0.000062,6.075810e-06,1.809800e-08,7.034950e-06,4.647920e-01,7.421230e-07
2,rt : are you another gardasil pusherman ?,gardasil,0.000124,0.000863,9.986260e-01,5.476730e-05,3.448030e-05,0.000057,3.026150e-05,3.472100e-06,2.307380e-05,1.556260e-04,2.886010e-05
3,hpv vaccination rates highest among hispanic g...,"vax, vaccin",0.000151,0.576814,1.727790e-04,1.593560e-04,2.323190e-05,0.422348,4.732910e-05,4.744580e-05,2.285810e-04,1.357940e-06,6.817710e-06
4,rt : online : integrated genomic and molecular...,cervicalcanc,0.000022,0.000391,9.919180e-06,2.330830e-05,3.723770e-05,0.000001,9.993910e-01,1.153040e-04,9.099290e-07,2.238840e-06,4.609420e-06


In [3]:
#add a columen with a list of combining all the probabilities
fields = list(df)[2:]
l = []
for i, row in df.iterrows():
    l.append(list(row[h] for h in fields))
df['p'] = pd.Series(l).values
df.head(5)

,tweet_text,true_label,health,vax,gardasil,hpvvaccin,cancer,vaccin,cervicalcanc,studi,learntherisk,hpv,vaccineswork,p
0,vaccine cuts teen girls ' hpv rates by two-thi...,health,0.997747,0.001123,6.564490e-07,2.698930e-07,3.316900e-07,0.001125,6.192670e-07,1.152250e-06,4.404900e-07,8.083010e-07,1.055320e-07,"[0.9977469999999999, 0.00112323, 6.56449000000..."
1,"rt : . vaxxed update ! gardasil tragedy , lead...",hpv,0.000021,0.000244,5.348350e-01,2.528250e-05,7.599710e-06,0.000062,6.075810e-06,1.809800e-08,7.034950e-06,4.647920e-01,7.421230e-07,"[2.05549e-05, 0.000243562, 0.5348350000000001,..."
2,rt : are you another gardasil pusherman ?,gardasil,0.000124,0.000863,9.986260e-01,5.476730e-05,3.448030e-05,0.000057,3.026150e-05,3.472100e-06,2.307380e-05,1.556260e-04,2.886010e-05,"[0.00012371799999999998, 0.0008626560000000001..."
3,hpv vaccination rates highest among hispanic g...,"vax, vaccin",0.000151,0.576814,1.727790e-04,1.593560e-04,2.323190e-05,0.422348,4.732910e-05,4.744580e-05,2.285810e-04,1.357940e-06,6.817710e-06,"[0.000150698, 0.576814, 0.00017277900000000002..."
4,rt : online : integrated genomic and molecular...,cervicalcanc,0.000022,0.000391,9.919180e-06,2.330830e-05,3.723770e-05,0.000001,9.993910e-01,1.153040e-04,9.099290e-07,2.238840e-06,4.609420e-06,"[2.24698e-05, 0.00039149800000000005, 9.91918e..."


In [4]:
data = np.array(df['p'])
data[:5]

array([list([0.9977469999999999, 0.00112323, 6.564490000000001e-07, 2.69893e-07, 3.3169000000000003e-07, 0.00112508, 6.19267e-07, 1.1522499999999999e-06, 4.4049e-07, 8.08301e-07, 1.0553200000000001e-07]),
       list([2.05549e-05, 0.000243562, 0.5348350000000001, 2.52825e-05, 7.59971e-06, 6.17787e-05, 6.07581e-06, 1.8098e-08, 7.03495e-06, 0.464792, 7.42123e-07]),
       list([0.00012371799999999998, 0.0008626560000000001, 0.998626, 5.476729999999999e-05, 3.44803e-05, 5.70348e-05, 3.02615e-05, 3.4720999999999998e-06, 2.3073800000000003e-05, 0.000155626, 2.8860100000000002e-05]),
       list([0.000150698, 0.576814, 0.00017277900000000002, 0.000159356, 2.32319e-05, 0.42234799999999995, 4.73291e-05, 4.74458e-05, 0.00022858099999999997, 1.3579399999999998e-06, 6.817710000000001e-06]),
       list([2.24698e-05, 0.00039149800000000005, 9.91918e-06, 2.33083e-05, 3.72377e-05, 1.4365100000000002e-06, 0.999391, 0.00011530399999999999, 9.09929e-07, 2.2388400000000003e-06, 4.609419999999999e-06])],